<a href="https://colab.research.google.com/github/Riz-waan/VIGORMM/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from music21 import converter, instrument, note, chord, midi, stream
import glob
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Dropout, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preprocessing
First we must load the data from the songs. To do this, we'll go through all the songs in our training data of MIDI files. We parse them with music21 to get the individual notes. If the element is a chord, then it is converted to it's numerical representation. After this step we will have all of the notes/chords that appear in string form, and a corresponding vocabulary as a set of them all.

In [4]:
notes = []
track = 0

for i, file in enumerate(glob.glob("drive/MyDrive/VIGOR/AE/music/*.mid")):
    midi = converter.parse(file)
    # There are multiple tracks in the MIDI file, so we'll use the first one
    midi = midi[track]
    notes_to_parse = None
        
    # Parse the midi file by the notes it contains
    notes_to_parse = midi.flat.notes
        
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            # get's the normal order (numerical representation) of the chord
            notes.append('.'.join(str(n) for n in element.normalOrder))
    print("Song {} Loaded".format(i+1))
                
print("DONE LOADING SONGS")    
# Get all pitch names
pitches = sorted(set(item for item in notes))
# Get all pitch names
vocab_length = len(pitches)  
number_notes = len(notes)
print(vocab_length)
print(notes)

Song 1 Loaded
Song 2 Loaded
DONE LOADING SONGS
152
['F6', 'D6', 'C#6', 'C6', 'B-5', 'G5', 'B-5', 'C6', 'C#6', 'C6', 'B-5', 'G5', 'F5', 'G5', '5.9', '7.10', '5.9', '7.10', 'C6', '5.9', '7.10', '5.9', '7.10', 'A5', 'G5', 'G6', 'F6', 'D6', 'C#6', 'C6', 'B-5', 'G5', 'B-5', 'C6', 'C#6', 'C6', 'B-5', 'G5', 'F5', 'G5', '5.9', '7.10', '5.9', '7.10', 'C6', '5.9', '7.10', '5.9', '7.10', 'A5', 'G5', 'C3', 'E-3', 'G#3', 'B-3', 'C4', 'G#3', 'E-4', 'B-4', 'C5', 'G#4', 'E-5', 'B-5', 'C6', 'G5', 'B-5', 'G#5', 'G5', 'F#5', 'F#5', 'G5', 'F#5', 'F5', 'F#5', 'A5', 'G5', 'E-5', 'E-5', 'D5', 'F5', 'E-5', 'D5', 'D5', 'C5', 'G4', 'B-4', 'C4', 'D4', 'F#4', 'B-4', 'A4', 'G4', '10.2', '10.2', 'D5', '2.7', '2.7', 'C5', '0.3.7', '0.3.7', 'C4', 'D4', 'F#4', 'B-4', 'A4', 'G4', '10.2', '10.2', 'E4', '7.9.1', '7.9.1', 'F#4', '2.6.9', '2.6.9', 'C4', 'D4', 'F#4', 'B-4', 'A4', 'G4', '10.2', '10.2', 'G5', '7.10.1', '7.10.1', 'F5', '10.0', '10.0', 'E-4', 'F4', 'A4', 'E-5', 'D5', 'C#5', 'C#5', '3.5.9', '3.5.9', 'D5', '10.2.

Now we must get these notes in a usable form for our LSTM. Let's construct sequences that can be grouped together to predict the next note in groups of 10 notes.

In [5]:
# Let's use One Hot Encoding for each of the notes and create an array as such of sequences. 
#Let's first assign an index to each of the possible notes
note_dict = dict()
for i, note in enumerate(pitches):
    note_dict[note] = i
#print(note_dict)

# Now let's construct sequences. Taking each note and encoding it as a numpy array with a 1 in the position of the note it has
sequence_length = 50
# Lets make a numpy array with the number of training examples, sequence length, and the length of the one-hot-encoding
num_training = number_notes - sequence_length

input_notes = np.zeros((num_training, sequence_length, vocab_length))
output_notes = np.zeros((num_training, vocab_length))

for i in range(0, num_training):
    # Here, i is the training example, j is the note in the sequence for a specific training example
    input_sequence = notes[i: i+sequence_length]
    output_note = notes[i+sequence_length]
    for j, note in enumerate(input_sequence):
        input_notes[i][j][note_dict[note]] = 1
    output_notes[i][note_dict[output_note]] = 1

print(len(input_notes[2049]))
print(len(input_notes[2049][49]))

# print(np.reshape(input_notes[0], (25, 25)))


50
152


In [30]:
import xarray as xr
import numpy as np


# Create a 10x10 array of random numbers


data = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

count = 0;

for x in input_notes:
  for y in x:
    
    count += 1
    j = np.array(y)
    j = np.append(j, np.zeros(10))
    a = xr.DataArray([j], dims=['x', 'y'])

    # "Downscale" the array, mean of blocks of size (2x2)
    b = a.coarsen(x=1, y=9).sum()
    t = np.array(b[0])
    data = np.append(data, [t], axis=0)
    if count >= 10000:
      break
  if count >= 10000:
      break
data = np.delete(data, 0, axis=0)



10000


In [34]:
print(len(data))
x = data[:5000]
print(len(x))
y = data[5000:10000]
print(len(y))
np.savetxt("drive/MyDrive/VIGOR/AE/CSVs/target.csv", x, delimiter=",")
np.savetxt("drive/MyDrive/VIGOR/AE/NW-CSVs/target-2.csv", y, delimiter=",")

10000
5000
5000
